In [3]:
import numpy as np
import random

In [4]:
def kernel_expo(x,y,c):
    return np.exp(-np.linalg.norm(x-y)**2/c)

In [5]:
def KKTviol(O,rho,mu,l,alpha,ens):
    ens = list(ens)
    random.shuffle(ens)
    for index in ens:
        if (O[index] -rho)*alpha[index]<=0 and (rho-O[index])*(1/(mu*l)-alpha[index])<=0:
            return index
    return -1

In [6]:
def argmax(O,i,ens):
    index = 0
    value = 0
    ens = list(ens)
    random.shuffle(ens)
    for j in ens:
        if value < abs(O[i]-O[j]):
            value = abs(O[i]-O[j])
            index = j
    return index    

In [42]:
def optim(alpha,i,j,K,O,mu,l):
    if K[i,i]+K[j,j]-2*K[i,j] == 0:
        return alpha[i],alpha[j]
    else:
        alphaj = alpha[j] + (O[i]-O[j])/(K[i,i]+K[j,j]-2*K[i,j])
        if alphaj < 0:
            alphaj = 0
        if alphaj >1/(l*mu):
            alphaj = 1/(l*mu)
        delta = alpha[i]+alpha[j]
        alphai = delta - alphaj
    print(alphai,alphaj)
    return alphai,alphaj
    

In [43]:
def SVcalc(alpha,c):
    SV = []
    for i in range(len(alpha)):
        if alpha[i]>0 and alpha[i]<c:
            SV.append(i)
    return SV

In [44]:
class osvm:
    def __init__(self,mu=0.7,maxiter=10000,eps=0.001):
        assert mu>0,"mu doit être strictement positif"
        assert mu<=1,"mu doit être inférieur à 1"
        self.mu = mu
        self.eps = eps
        self.maxiter = maxiter
        
    def fit(self,data):
        self.alpha = np.zeros(np.size(data))
        l = np.size(data)        
        K = np.array([[kernel_expo(x,y,0.5*l) for x in data] for y in data])
        choose = [i for i in range(l)]
        self.rho = 0
        s = 0
        while s <= 1-1/(l*self.mu):
            index = np.random.randint(len(choose))
            elem = choose[index]
            choose.remove(elem)
            self.alpha[elem] = 1/(l*self.mu)
            s += 1/(l*self.mu)
        if s != 1:
            index = np.random.randint(len(choose))
            elem = choose[index]
            choose.remove(elem)
            self.alpha[elem] = 1-s
            print(1-s)
            s = 1
        C = np.dot(self.alpha[2:],K[2:,:])
        O = K[0,:]*self.alpha[0] + K[1,:]*self.alpha[1] + C
        for i in range(l):
            if self.alpha[i]>0:
                self.rho=max(self.rho,O[i])

        iteration = 0
        while iteration < self.maxiter:
            # Etape (i)
            index = KKTviol(O,self.rho,self.mu,l,self.alpha,range(l))
            if index == -1:
                break
            iteration += 1
            if iteration == 0 :
                SV = []
                for i in range(l):
                    if self.alpha[i] > 0:
                        SV.append(i)
            else:
                SV = SVcalc(self.alpha,1/(self.mu*l))
            if SV == []:
                print("out")
                break
            
            j = argmax(O,index,SV)
            self.alpha[index],self.alpha[j] = optim(self.alpha,index,j,K,O,self.mu,l)
            
            # update de rho
            O = np.dot(self.alpha,K)
            for i in range(l):
                if self.alpha[i]>0:
                    self.rho=max(self.rho,O[i])
            SV = SVcalc(self.alpha,1/(self.mu*l))
            
            # Etape (ii)
            index = KKTviol(O,self.rho,self.mu,l,self.alpha,SV)
            while index !=-1:
                j = argmax(O,index,SV)
                self.alpha[index],self.alpha[j] = optim(self.alpha,index,j,K,O,self.mu,l)
                # update de rho
                O = np.dot(self.alpha,K)
                for i in range(l):
                    if self.alpha[i]>0:
                        self.rho=max(self.rho,O[i])
                print(index)
                SV = SVcalc(self.alpha,1/(self.mu*l))
                index = KKTviol(O,self.rho,self.mu,l,self.alpha,SV)
            
            if iteration%1 == 0 :
                print(iteration,self.maxiter)
            
            iteration += 1
                
        return self.alpha,self.rho        

In [45]:
test = osvm(1/3.14)
data = np.random.rand(100)
test.fit(data)

0.02660000000000029
0.0266 0.031400000000000004
1 10000
0.058 0
3 10000
out


(array([ 0.    ,  0.    ,  0.0314,  0.    ,  0.0314,  0.0314,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.0314,  0.    ,  0.    ,  0.    ,  0.0314,  0.0314,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.058 ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.0314,  0.0314,  0.    ,  0.    ,
         0.0314,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.0314,  0.0314,  0.0314,  0.    ,  0.    ,
         0.0314,  0.    ,  0.    ,  0.    ,  0.0314,  0.    ,  0.    ,
         0.    ,  0.    ,  0.0314,  0.0314,  0.0314,  0.0314,  0.0314,
         0.0314,  0.    ,  0.    ,  0.0314,  0.0314,  0.0314,  0.    ,
         0.    ,  0.    ,  0.0314,  0.0314,  0.0314,  0.    ,  0.    ,
         0.    ,  0.    ,  0.0314,  0.    ,  0.    ,  0.    ,  0.    ,
         0.0314,  0.    ,  0.    ,  0.0314,  0.    ,  0.    ,  0.0314,
      

In [39]:
# probleme de la projection de alpha1 et alpha2


1.0